In [5]:
import requests

# URL of audio key
audio_url = "https://audio-lingua.ac-versailles.fr/IMG/mp3/gettysburg_address.mp3"
audio_filename = "audio_key.mp3"

# Download the audio file
print("Downloading audio...")
resp = requests.get(audio_url)
with open(audio_filename, "wb") as f:
    f.write(resp.content)
print(f"Saved audio file as {audio_filename}")


Saved audio file as audio_key.mp3


In [10]:
# Install and import pocketsphinx (for offline speech recognition)
# %pip install pocketsphinx  # (If not already installed)

from pocketsphinx import Decoder, get_model_path

# Configure PocketSphinx for phoneme recognition
model_dir = get_model_path()                     # default model directory path
hmm_dir   = model_dir + "/en-us"                 # acoustic model (English)
phoneme_lm = model_dir + "/en-us/en-us-phone.lm.bin"  # phonetic language model

config = Decoder.default_config()
config.set_string('-hmm', hmm_dir)
config.set_string('-allphone', phoneme_lm)       # enable phoneme decoding mode:contentReference[oaicite:9]{index=9}
config.set_boolean('-allphone_ci', True)         # use context-independent phonemes for simplicity

decoder = Decoder(config)

# Convert MP3 to WAV (16kHz mono) if needed
# (Requires pydub or ffmpeg; this step ensures the audio is in proper PCM format)
try:
    from pydub import AudioSegment
    sound = AudioSegment.from_mp3(audio_filename)
    sound = sound.set_frame_rate(16000).set_channels(1)  # 16 kHz mono
    wav_filename = "gettysburg_address.wav"
    sound.export(wav_filename, format="wav")
    
except Exception as e:
    print("Conversion to WAV might require ffmpeg. Proceeding with raw decoding of MP3.")
    wav_filename = audio_filename

# Perform speech decoding to phonemes
decoder.start_utt()
with open(wav_filename, "rb") as audio_file:
    data = audio_file.read()
    decoder.process_raw(data, False, True)
decoder.end_utt()

# Retrieve the phonetic sequence
phonemes = []
# PocketSphinx segments (decoder.seg()) will iterate over recognized phoneme segments
for seg in decoder.seg():
    phonemes.append(seg.word)  # e.g., 'F', 'AO', 'R', 'S', 'K', 'AO', 'R', ... 'SIL'
# Filter out silence markers
phonetic_sequence = [p for p in phonemes if p != "<sil>" and p.upper() != "SIL"]
print("Phonetic transcription:", " ".join(phonetic_sequence))


C:\Users\chiri\AppData\Local\Temp\ipykernel_14908\2704875400.py:11: DeprecationWarning: default_config() is deprecated, just call Config() constructor
  config = Decoder.default_config()


RuntimeError: Failed to initialize PocketSphinx

In [ ]:
import pronouncing
import re

# Load or define the expected text of the Gettysburg Address (English version)
actual_text = """Four score and seven years ago our fathers brought forth on this continent, 
a new nation, conceived in Liberty, and dedicated to the proposition that all men are created equal. 
Now we are engaged in a great civil war, testing whether that nation, or any nation so conceived and so dedicated, 
can long endure. We are met on a great battle-field of that war. We have come to dedicate a portion of that field, 
as a final resting place for those who here gave their lives that that nation might live. 
It is altogether fitting and proper that we should do this. 
But, in a larger sense, we can not dedicate -- we can not consecrate -- we can not hallow -- this ground. 
The brave men, living and dead, who struggled here, have consecrated it far above our poor power to add or detract. 
The world will little note, nor long remember what we say here, but it can never forget what they did here. 
It is for us the living, rather, to be dedicated here to the unfinished work which they who fought here have thus far so nobly advanced. 
It is rather for us to be here dedicated to the great task remaining before us -- 
that from these honored dead we take increased devotion to that cause for which they gave the last full measure of devotion -- 
that we here highly resolve that these dead shall not have died in vain -- 
that this nation, under God, shall have a new birth of freedom -- 
and that government of the people, by the people, for the people, shall not perish from the earth."""
# Normalize the text: remove punctuation and lowercase it for matching
normalized_text = re.sub(r'[^A-Za-z\s]', ' ', actual_text)  # remove punctuation
normalized_text = re.sub(r'\s+', ' ', normalized_text).strip().lower()

# Build a lexicon: dictionary mapping each word to its phoneme sequence (as a list of symbols)
lexicon = {}
for word in normalized_text.split():
    if word not in lexicon:  # avoid duplicates
        phones_list = pronouncing.phones_for_word(word)
        if phones_list:
            # Take the first pronunciation and remove any stress numbers (1, 0, 2)
            phones = phones_list[0].split()
            phones = [ph[:-1] if ph[-1].isdigit() else ph for ph in phones]
            lexicon[word] = phones
        else:
            lexicon[word] = []  # word not found in dictionary (if any)

# Now, perform a search to map the phonetic_sequence to words
def phoneme_seq_to_text(seq, idx=0):
    """Recursive DFS to match phoneme sequence to words from lexicon."""
    if idx == len(seq):
        return []  # reached end successfully
    for word, phones in lexicon.items():
        L = len(phones)
        if L > 0 and idx + L <= len(seq) and seq[idx:idx+L] == phones:
            result = phoneme_seq_to_text(seq, idx+L)
            if result is not None:  # found a valid continuation
                return [word] + result
    return None

# Convert the phonetic sequence to text
recognized_words = phoneme_seq_to_text(phonetic_sequence)
recognized_text = " ".join(recognized_words) if recognized_words else ""
print("Reconstructed text from phonemes:\n", recognized_text)


In [ ]:
# Install and import pocketsphinx (for offline speech recognition)
# %pip install pocketsphinx  # (If not already installed)

from pocketsphinx import Decoder, get_model_path

# Configure PocketSphinx for phoneme recognition
model_dir = get_model_path()                     # default model directory path
hmm_dir   = model_dir + "/en-us"                 # acoustic model (English)
phoneme_lm = model_dir + "/en-us/en-us-phone.lm.bin"  # phonetic language model

config = Decoder.default_config()
config.set_string('-hmm', hmm_dir)
config.set_string('-allphone', phoneme_lm)       # enable phoneme decoding mode:contentReference[oaicite:9]{index=9}
config.set_boolean('-allphone_ci', True)         # use context-independent phonemes for simplicity

decoder = Decoder(config)

# Convert MP3 to WAV (16kHz mono) if needed
# (Requires pydub or ffmpeg; this step ensures the audio is in proper PCM format)
try:
    from pydub import AudioSegment
    sound = AudioSegment.from_mp3(audio_filename)
    sound = sound.set_frame_rate(16000).set_channels(1)  # 16 kHz mono
    wav_filename = "gettysburg_address.wav"
    sound.export(wav_filename, format="wav")
    
except Exception as e:
    print("Conversion to WAV might require ffmpeg. Proceeding with raw decoding of MP3.")
    wav_filename = audio_filename

# Perform speech decoding to phonemes
decoder.start_utt()
with open(wav_filename, "rb") as audio_file:
    data = audio_file.read()
    decoder.process_raw(data, False, True)
decoder.end_utt()

# Retrieve the phonetic sequence
phonemes = []
# PocketSphinx segments (decoder.seg()) will iterate over recognized phoneme segments
for seg in decoder.seg():
    phonemes.append(seg.word)  # e.g., 'F', 'AO', 'R', 'S', 'K', 'AO', 'R', ... 'SIL'
# Filter out silence markers
phonetic_sequence = [p for p in phonemes if p != "<sil>" and p.upper() != "SIL"]
print("Phonetic transcription:", " ".join(phonetic_sequence))
